In [234]:
#Import dependencies.
import pandas as pd
import os
import csv
import geopandas as gpd
from shapely.geometry import Point

# Population Data Cleaning.

In [235]:
#Read dataframe.
conflict_df=pd.read_csv("Resources/cleaned conflict.csv",header=0)
demographics_df=pd.read_csv("Resources/demographics.csv",header=0)
disasters_df=pd.read_csv("Resources/cleaned disasters.csv",header=0)
decision_df=pd.read_csv("Resources/cleaned_asylum.csv",header=0)

In [236]:
country_list = demographics_df['Country of origin'].unique().tolist()

In [237]:
conflict_df['helper'] = conflict_df['Year'].astype(str) + conflict_df['Country of origin']
demographics_df['helper'] = demographics_df['Year'].astype(str) + demographics_df['Country of origin']
disasters_df['helper'] = disasters_df['Year'].astype(str) + disasters_df['Country of origin']
decision_df['helper'] = decision_df['Year'].astype(str) + decision_df['Country of origin']
demographics_df

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,...,Female total,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Male total,Total,helper
0,2001,Iran (Islamic Rep. of),IRN,Afghanistan,AFG,0,0,0,0,0,...,0,0,0,0,5,0,0,5,5,2001Iran (Islamic Rep. of)
1,2001,Iraq,IRQ,Afghanistan,AFG,0,5,0,0,0,...,5,0,0,0,0,0,0,0,5,2001Iraq
2,2001,Iraq,IRQ,Albania,ALB,0,0,0,0,0,...,0,0,0,0,7,0,0,7,7,2001Iraq
3,2001,North Macedonia,MKD,Albania,ALB,0,0,0,6,0,...,6,0,0,0,0,0,0,0,6,2001North Macedonia
4,2001,Serbia and Kosovo: S/RES/1244 (1999),SRB,Albania,ALB,14,20,18,89,8,...,149,14,18,16,45,18,0,111,278,2001Serbia and Kosovo: S/RES/1244 (1999)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74568,2021,Rwanda,RWA,Zimbabwe,ZWE,29,48,49,179,14,...,319,25,54,51,188,23,0,341,660,2021Rwanda
74569,2021,Somalia,SOM,Zimbabwe,ZWE,0,0,0,0,0,...,0,0,0,0,13,0,0,13,13,2021Somalia
74570,2021,Sudan,SDN,Zimbabwe,ZWE,0,0,0,0,0,...,0,0,0,0,8,0,0,8,8,2021Sudan
74571,2021,South Sudan,SSD,Zimbabwe,ZWE,0,0,0,0,0,...,0,0,0,0,8,0,0,8,8,2021South Sudan


In [238]:
# Group the data by Country of origin and Year and calculate the sum of the Total
summed_df = demographics_df.groupby(['Country of origin','Country of origin (ISO)', 'Country of asylum', 'Country of asylum (ISO)','Year', 'helper', 'Female 0 - 4', 'Female 5 - 11', 'Female 12 - 17', 'Female 18 - 59', 'Female 60', 'Female other', 'Female total', 'Male 0 - 4', 'Male 5 - 11', 'Male 12 - 17', 'Male 18 - 59', 'Male 60', 'Male other', 'Male total'], as_index=False)['Total'].sum()

# Group by Country of origin to get the full count of every year and drop if the Total count is below 50,000
filtered_df = summed_df.groupby('Country of origin').filter(lambda x: x['Total'].sum() >= 50000)

# Reset the index to ungroup the data
demo_result_df = pd.DataFrame(filtered_df.reset_index(drop=True))

In [239]:
grouped_conflict_df = conflict_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_demo_result_df = demo_result_df.groupby(['Year', 'Country of origin','Country of origin (ISO)','Country of asylum','Country of asylum (ISO)','helper']).sum()
grouped_disasters_df = disasters_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_decision_df = decision_df.groupby(['Year', 'Country of origin','Country of origin (ISO)', 'Country of asylum','Country of asylum (ISO)','helper']).sum()
grouped_conflict_df = pd.DataFrame(grouped_conflict_df.reset_index())
grouped_demo_result_df = pd.DataFrame(grouped_demo_result_df.reset_index())
grouped_disasters_df = pd.DataFrame(grouped_disasters_df.reset_index())
grouped_decision_df = pd.DataFrame(grouped_decision_df.reset_index())
demo_result_df

,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Year,helper,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,...,Female other,Female total,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Male total,Total
0,Afghanistan,AFG,Albania,ALB,2021,2021Afghanistan,0,5,0,0,...,0,5,0,0,0,0,0,0,0,5
1,Afghanistan,AFG,Argentina,ARG,2014,2014Afghanistan,0,0,0,0,...,0,0,0,5,0,0,0,0,5,5
2,Afghanistan,AFG,Argentina,ARG,2015,2015Afghanistan,0,0,0,0,...,0,0,0,5,0,0,0,0,5,5
3,Afghanistan,AFG,Argentina,ARG,2016,2016Afghanistan,0,0,0,0,...,0,0,0,5,0,0,0,0,5,5
4,Afghanistan,AFG,Argentina,ARG,2017,2017Afghanistan,0,0,0,0,...,0,0,0,5,0,0,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,Zimbabwe,ZWE,Zambia,ZMB,2013,2013Zimbabwe,0,0,0,0,...,0,0,0,0,6,0,0,0,6,6
61195,Zimbabwe,ZWE,Zambia,ZMB,2014,2014Zimbabwe,0,0,0,0,...,0,0,0,0,6,0,0,0,6,6
61196,Zimbabwe,ZWE,Zambia,ZMB,2015,2015Zimbabwe,0,0,0,0,...,0,0,0,0,6,0,0,0,6,6
61197,Zimbabwe,ZWE,Zambia,ZMB,2016,2016Zimbabwe,0,0,0,0,...,0,0,0,0,7,0,0,0,7,7


In [240]:
new_country_list = grouped_demo_result_df['Country of origin'].unique().tolist()
print(len(new_country_list))

87


In [241]:
dropped_countries = list(set(country_list) - set(new_country_list))
dropped_countries.sort()
print(len(dropped_countries))

114


In [242]:

filtered_decision_df = grouped_decision_df[['Year','Country of origin','Country of origin (ISO)','Country of asylum','Country of asylum (ISO)','helper','Recognized decisions','Complementary protection','Rejected decisions','Otherwise closed', 'Total decisions']]
filtered_demographics_df = grouped_demo_result_df[['Year','Country of origin', 'Country of origin (ISO)', 'Country of asylum', 'Country of asylum (ISO)','helper','Female 0 - 4','Female 5 - 11','Female 12 - 17','Female 18 - 59','Female 60','Female other','Male 0 - 4','Male 5 - 11','Male 12 - 17','Male 18 - 59','Male 60','Male other','Total']]
filtered_conflict_df = grouped_conflict_df[['Year','Country of origin','helper', 'Deaths civilians']]
filtered_disasters_df = grouped_disasters_df[['Year','Country of origin','helper','Total deaths','Total affected']]
filtered_demographics_df

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),helper,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Total
0,2001,Afghanistan,AFG,Australia,AUS,2001Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,6262
1,2001,Afghanistan,AFG,Austria,AUT,2001Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,1049
2,2001,Afghanistan,AFG,Azerbaijan,AZE,2001Afghanistan,8,12,10,38,0,0,9,26,22,118,0,0,243
3,2001,Afghanistan,AFG,Belarus,BLR,2001Afghanistan,9,40,36,81,6,0,9,34,35,199,5,0,454
4,2001,Afghanistan,AFG,Belgium,BEL,2001Afghanistan,0,0,0,0,0,144,0,0,0,0,0,213,357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,2021,Zimbabwe,ZWE,Sweden,SWE,2021Zimbabwe,0,0,0,0,0,0,0,0,0,0,0,0,16
61195,2021,Zimbabwe,ZWE,Switzerland,CHE,2021Zimbabwe,0,9,0,0,0,0,0,0,0,0,0,0,9
61196,2021,Zimbabwe,ZWE,Thailand,THA,2021Zimbabwe,0,0,0,0,0,0,9,0,0,0,0,0,9
61197,2021,Zimbabwe,ZWE,United Kingdom of Great Britain and Northern I...,GBR,2021Zimbabwe,0,0,0,0,0,0,0,0,0,0,0,0,1174


In [243]:
filtered_conflict_df = filtered_conflict_df.rename(columns={'Deaths civilians': 'conflict_deaths'})
filtered_conflict_df

,Year,Country of origin,helper,conflict_deaths
0,2001,Afghanistan,2001Afghanistan,864
1,2001,Algeria,2001Algeria,32
2,2001,Angola,2001Angola,1038
3,2001,Azerbaijan,2001Azerbaijan,0
4,2001,Bangladesh,2001Bangladesh,6
...,...,...,...,...
1092,2021,Turkey,2021Turkey,0
1093,2021,Uganda,2021Uganda,7
1094,2021,Ukraine,2021Ukraine,18
1095,2021,Venezuela,2021Venezuela,19


In [244]:
filtered_disasters_df = filtered_disasters_df.rename(columns={'Total deaths': 'disaster_deaths', 'Total affected':'disaster_affected'})
filtered_disasters_df 

,Year,Country of origin,helper,disaster_deaths,disaster_affected
0,2001,Afghanistan,2001Afghanistan,331.0,200270.0
1,2001,Algeria,2001Algeria,921.0,45423.0
2,2001,Angola,2001Angola,48.0,39928.0
3,2001,Argentina,2001Argentina,15.0,253000.0
4,2001,Australia,2001Australia,3.0,3001.0
...,...,...,...,...,...
1477,2021,United States of America,2021United States of America,494.0,17059.0
1478,2021,Uzbekistan,2021Uzbekistan,8.0,6.0
1479,2021,Viet Nam,2021Viet Nam,9.0,83430.0
1480,2021,Yemen,2021Yemen,13.0,22380.0


In [245]:
#filtered_demographics_df

#adding all the male and female by age totals. 
filtered_demographics_df['MaleFemaleTotal'] = filtered_demographics_df[['Female 0 - 4','Female 5 - 11','Female 12 - 17','Female 18 - 59','Female 60','Female other','Male 0 - 4','Male 5 - 11','Male 12 - 17','Male 18 - 59','Male 60','Male other']].sum(axis=1)
#creating an Unknown demographic calculated field. 
filtered_demographics_df['unknown_demographic'] = filtered_demographics_df['Total'] - filtered_demographics_df['MaleFemaleTotal']
col_to_move = filtered_demographics_df.pop('unknown_demographic')  
filtered_demographics_df.insert(16, 'unknown_demographic', col_to_move)
filtered_demographics_df = filtered_demographics_df.drop('helper', axis=1)
filtered_demographics_df = filtered_demographics_df.drop('MaleFemaleTotal', axis=1)
filtered_demographics_df=filtered_demographics_df.replace(to_replace="TÃ¼rkiye",
           value="Turkey")
demographic_ml_df = filtered_demographics_df
demographic_ml_df 
demographic_ml_df = pd.merge(demographic_ml_df, filtered_decision_df , on=['Year', 'Country of origin','Country of origin (ISO)', 'Country of asylum','Country of asylum (ISO)',], how='left')

In [246]:
demographic_ml_df

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,...,unknown_demographic,Male 60,Male other,Total,helper,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions
0,2001,Afghanistan,AFG,Australia,AUS,0,0,0,0,0,...,6262,0,0,6262,2001Afghanistan,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,AFG,Austria,AUT,0,0,0,0,0,...,1049,0,0,1049,2001Afghanistan,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,AFG,Azerbaijan,AZE,8,12,10,38,0,...,0,0,0,243,2001Afghanistan,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,AFG,Belarus,BLR,9,40,36,81,6,...,0,5,0,454,2001Afghanistan,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,AFG,Belgium,BEL,0,0,0,0,0,...,0,0,213,357,2001Afghanistan,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,2021,Zimbabwe,ZWE,Sweden,SWE,0,0,0,0,0,...,16,0,0,16,2021Zimbabwe,0.0,0.0,16.0,0.0,16.0
61195,2021,Zimbabwe,ZWE,Switzerland,CHE,0,9,0,0,0,...,0,0,0,9,2021Zimbabwe,0.0,0.0,5.0,0.0,5.0
61196,2021,Zimbabwe,ZWE,Thailand,THA,0,0,0,0,0,...,0,0,0,9,NaN,NaN,NaN,NaN,NaN,NaN
61197,2021,Zimbabwe,ZWE,United Kingdom of Great Britain and Northern I...,GBR,0,0,0,0,0,...,1174,0,0,1174,2021Zimbabwe,44.0,5.0,53.0,10.0,112.0


In [247]:

# Load the "world" dataset as a GeoDataFrame
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [248]:
def share_borders(country1, country2):
    try:
        # Get the geometry of country1
        geom1 = world.loc[world['name'] == country1, 'geometry'].iloc[0]

        # Get the geometry of country2
        geom2 = world.loc[world['name'] == country2, 'geometry'].iloc[0]

        # Check if the geometries intersect
        return geom1.intersects(geom2)
    except IndexError:
        # If either country is not found in the GeoDataFrame, return False
        return False

shares_border = []
# Loop through each row of the DataFrame
for index, row in demographic_ml_df.iterrows():
    origin = row['Country of origin']
    asylum = row['Country of asylum']
    
    # Check if the two countries share a border
    if share_borders(origin, asylum):
        print(f"{origin} shares a border with {asylum}")
        shares_border.append('yes')
    else:
        print(f"{origin} DOES NOT shares a border with {asylum}")
        shares_border.append('no')

demographic_ml_df['share_borders'] = shares_border

#drop the ISO columns
demographic_ml_df = demographic_ml_df.drop('Country of origin (ISO)', axis=1)
demographic_ml_df = demographic_ml_df.drop('Country of asylum (ISO)', axis=1)

#move the new shared border column next to the country of origin and country of asylum
col_to_move = demographic_ml_df.pop('share_borders')  
demographic_ml_df.insert(3, 'share_borders', col_to_move)

Afghanistan DOES NOT shares a border with Australia
Afghanistan DOES NOT shares a border with Austria
Afghanistan DOES NOT shares a border with Azerbaijan
Afghanistan DOES NOT shares a border with Belarus
Afghanistan DOES NOT shares a border with Belgium
Afghanistan DOES NOT shares a border with Bulgaria
Afghanistan DOES NOT shares a border with Cambodia
Afghanistan DOES NOT shares a border with Canada
Afghanistan DOES NOT shares a border with Chile
Afghanistan DOES NOT shares a border with Cote d'Ivoire
Afghanistan DOES NOT shares a border with Cuba
Afghanistan DOES NOT shares a border with Cyprus
Afghanistan DOES NOT shares a border with Czechia
Afghanistan DOES NOT shares a border with Denmark
Afghanistan DOES NOT shares a border with Ecuador
Afghanistan DOES NOT shares a border with Egypt
Afghanistan DOES NOT shares a border with Estonia
Afghanistan DOES NOT shares a border with Ethiopia
Afghanistan DOES NOT shares a border with Finland
Afghanistan DOES NOT shares a border with Fra

Determine if a country intersects with another country. 

In [249]:
#rename all the columns to best practing naming conventions 

demographic_ml_df = demographic_ml_df.rename(columns={'Year' : 'year',
                                                    'Country of origin' : 'country_origin',	
                                                    'Country of asylum' : 'country_asylum',	
                                                    'Female 0 - 4' : 'female_0to4',	
                                                    'Female 5 - 11' : 'female_5to11',	
                                                    'Female 12 - 17' : 'female_12to17',	
                                                    'Female 18 - 59' : 'female_18to59',	
                                                    'Female 60' :	'female_60',
                                                    'Female other' : 'female_other',	
                                                    'Male 0 - 4' : 'male_0to4',		
                                                    'Male 5 - 11' : 'male_5to11',
                                                    'Male 12 - 17' : 'male_12to17',	
                                                    'Male 18 - 59' : 'male_18to59',	
                                                    'Male 60' : 'male_60',	
                                                    'Male other' : 'male_other',
                                                    'Total' : 'total',
                                                    'Recognized decisions':'recognized_decisions',
                                                    'Complementary protection':'complementary_protection',
                                                    'Rejected decisions':'rejected',
                                                    'Otherwise closed':'otherwise_closed',
                                                    'Total decisions':'total_decisions'
    
})
demographic_ml_df

,year,country_origin,country_asylum,share_borders,female_0to4,female_5to11,female_12to17,female_18to59,female_60,female_other,...,unknown_demographic,male_60,male_other,total,helper,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,2001,Afghanistan,Australia,no,0,0,0,0,0,0,...,6262,0,0,6262,2001Afghanistan,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,Austria,no,0,0,0,0,0,0,...,1049,0,0,1049,2001Afghanistan,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,Azerbaijan,no,8,12,10,38,0,0,...,0,0,0,243,2001Afghanistan,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,Belarus,no,9,40,36,81,6,0,...,0,5,0,454,2001Afghanistan,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,Belgium,no,0,0,0,0,0,144,...,0,0,213,357,2001Afghanistan,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61194,2021,Zimbabwe,Sweden,no,0,0,0,0,0,0,...,16,0,0,16,2021Zimbabwe,0.0,0.0,16.0,0.0,16.0
61195,2021,Zimbabwe,Switzerland,no,0,9,0,0,0,0,...,0,0,0,9,2021Zimbabwe,0.0,0.0,5.0,0.0,5.0
61196,2021,Zimbabwe,Thailand,no,0,0,0,0,0,0,...,0,0,0,9,NaN,NaN,NaN,NaN,NaN,NaN
61197,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,no,0,0,0,0,0,0,...,1174,0,0,1174,2021Zimbabwe,44.0,5.0,53.0,10.0,112.0


In [250]:
#data merged by year totals. 

merged = pd.merge(filtered_conflict_df, filtered_disasters_df , on=['Year', 'Country of origin','helper'], how='left')
merged = merged.drop('helper', axis=1)
col_to_move = merged.pop('conflict_deaths')  
merged.insert(4, 'conflict_deaths', col_to_move)
for col in merged.columns:
    if merged[col].dtype == float:
        merged[col] = merged[col].fillna(0).astype(int)
event_ml_df = merged
event_ml_df = event_ml_df.rename(columns={'Year':'year',
                                          'Country of origin': 'country_origin',
                                        })
event_ml_df

,year,country_origin,disaster_deaths,disaster_affected,conflict_deaths
0,2001,Afghanistan,331,200270,864
1,2001,Algeria,921,45423,32
2,2001,Angola,48,39928,1038
3,2001,Azerbaijan,0,0,0
4,2001,Bangladesh,232,530150,6
...,...,...,...,...,...
1092,2021,Turkey,9,561088,0
1093,2021,Uganda,0,0,7
1094,2021,Ukraine,1,1700,18
1095,2021,Venezuela,0,0,19


In [252]:
demographic_ml_df.to_csv('Resources/demographic_ml_df.csv')
event_ml_df.to_csv('Resources/event_ml_df.csv')